# Convert postISRCCD into sources

 work with Weakly_2022_39
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/10/31
- update : 2022/11/01


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import lsst.daf.butler as dafButler
import lsst.daf.base as dafBase

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

# Pipeline tasks from DP0.2
#from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
#from lsst.meas.algorithms.detection import SourceDetectionTask
#from lsst.meas.deblender import SourceDeblendTask
#from lsst.meas.base import SingleFrameMeasurementTask

# fromm StackClub on Britter-Fatter
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
import lsst.meas.extensions.shapeHSM
from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig

In [ ]:
#Set the matplotlib plot color table.
plt.style.use('tableau-colorblind10')

In [ ]:
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
afwDisplay.setDefaultBackend('matplotlib')
plt.rcParams['figure.figsize'] = (6.0, 6.0)

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

## Give the collection

In [ ]:
my_collection = "auxtel_atmosphere_202301_v3.0.3_doGainsPTC_rebin" 

In [ ]:
#butler = butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection])

## Config

In [ ]:
# path index for each month
DATE="20231207"
filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='icSrc_schema', collections=my_collection, where= "instrument='LATISS'")
#icSrc_schema = butler.get('icSrc_schema')

## postISRCCD

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='postISRCCD', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
all_dataId = []
all_postisrccd  = []
all_exposures = []



for i, ref in enumerate(datasetRefs):
   
    print(f"========({i})================datasetType = postISRCCD ============================================")
    print("fullId..................:",ref.dataId.full)
    print("exposure................:",ref.dataId["exposure"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_exposure = ref.dataId["exposure"]
    the_day_obs = ref.dataId["exposure"]//100_000
    the_seq_num = ref.dataId["exposure"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0}
    print(the_dataId)
    #spec       = butler.get('spectraction',the_dataId)
    postisrccd = butler.get('postISRCCD', exposure=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_postisrccd.append(postisrccd)
    if i > 0:
        break

In [ ]:
all_exposures

In [ ]:
# Plot the calexp we just retrieved
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
title = 'postISRCCD image' + str(all_exposures[-1]) 
afw_display.mtv(postisrccd.image,title=title)

In [ ]:
# Plot the calexp we just retrieved
plt.figure()
afw_display = afwDisplay.Display()
afw_display.scale('asinh', 'zscale')
title = 'postISRCCD masked image' + str(all_exposures[-1]) 
afw_display.mtv(postisrccd.maskedImage.image,title=title)

In [ ]:
postisrccd_md=dict(postisrccd.getMetadata())

In [ ]:
ccd_ymax=postisrccd.getBBox().getMaxY()
ccd_xmax=postisrccd.getBBox().getMaxX()

In [ ]:
postisrccd.image.array.flatten()

In [ ]:
fig=plt.figure(figsize=(6,4))
ax=fig.add_subplot(1,1,1)
ax.hist(postisrccd.image.array.flatten(),bins=100,range=(-50,1000),alpha=0.5,facecolor='blue')
ax.hist(postisrccd.maskedImage.image.array.flatten(),bins=100,range=(-50,1000),alpha=0.5,facecolor="yellow")
ax.set_yscale('log')
ax.grid()
ax.set_title("distribution of pixel value in postISRCCD")

# Detection of Sources

## Step 2: Perform image characterization and initial measurement
We now perform a base-level characterization of the image using the stack. We set some configuration settings which are specific to our sestup which has a very small optical PSF, setting a PSF size and turning off some other aspects such as cosmic ray rejection because of this.

In [ ]:
#from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig
#import lsst.meas.extensions.shapeHSM

# first set a few configs that are specific to our beam simulator data
charConfig = CharacterizeImageConfig()
#this set the fwhm of the simple PSF to that of optics
charConfig.installSimplePsf.fwhm = 10
charConfig.doMeasurePsf = False
charConfig.doApCorr = False # necessary
charConfig.repair.doCosmicRay = False  
# we do have some cosmic rays, but we also have subpixel mask features and an undersampled PSF
charConfig.detection.background.binSize = 10   # worth playing around with
#charConfig.background.binSize = 50
charConfig.detection.minPixels = 10   # also worth playing around with

# Add the HSM (Hirata/Seljak/Mandelbaum) adaptive moments shape measurement plugin
charConfig.measurement.plugins.names |= ["ext_shapeHSM_HsmSourceMoments"]
# to configure hsm you would do something like
# charConfig.measurement.plugins["ext_shapeHSM_hsmSourceMoments"].addFlux = True
# (see sfm.py in meas_base for all the configuration options for the measurement task)

# Turn off the summary statistic calculation for these spot images.
# In the future there may be a specialized task for spot images.
charConfig.doComputeSummaryStats = False

charTask = CharacterizeImageTask(config=charConfig)

#charTask.run?
# use charTask.run instead of characterize for v16.0+22
# could also perform similar functions with processCcdTask.run()

In [ ]:
# Display which plugins are being used for measurement
charConfig.measurement.plugins.active 

In [ ]:
charResult = charTask.run(postisrccd) # charTask.run(exposure) stack v16.0+22
print("Detected ",len(charResult.sourceCat)," objects ")
fig=plt.figure(figsize=(5,5))
plt.title('X/Y locations of detections')
plt.plot(charResult.sourceCat['base_SdssCentroid_x'],charResult.sourceCat['base_SdssCentroid_y'],'r.')
plt.grid()
plt.xlim(0,ccd_xmax)
plt.ylim(0,ccd_ymax)

This figure illustrates the centroids of detections made during characterization. Note that not all objects have been detected in this first round.

In [ ]:
charResult.sourceCat.asAstropy()

# Calibration and Photometry

## Step 3: Further image calibration and measurement
This builds on the exposure output from characterization, using the new mask plane as well as the source catalog. Similar to the characterization, we turn off some processing which is suited to our particular setup.  It does provide a background-subtracted image and for completeness it is included here. The steps in calibration that are turned on/off can be seen by printing the calibration config object.

In [ ]:
#from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig

calConfig = CalibrateConfig()
calConfig.doAstrometry = False
calConfig.doPhotoCal = False
calConfig.doApCorr = False
calConfig.doDeblend = False   # these are well-separated objects, deblending adds time & trouble
# these images should have a uniform background, so measure it
#  on scales which are larger than the objects
calConfig.detection.background.binSize = 50
calConfig.detection.minPixels = 5
calConfig.measurement.plugins.names |= ["ext_shapeHSM_HsmSourceMoments"]
# to configure hsm you would do something like
#charConfig.measurement.plugins["ext_shapeHSM_hsmSourceMoments"].addFlux = True
calConfig.doComputeSummaryStats = True # no summary stats

calTask = CalibrateTask(config= calConfig, icSourceSchema=charResult.sourceCat.schema)

#calTask.run? # for stack v16.0+22 
calTask.run?

In [ ]:
# for stack v16.0+22, change to calTask.run(charResult.exposure)
calResult = calTask.run(charResult.exposure, background=charResult.background,
                              icSourceCat = charResult.sourceCat)


print("Detected ",len(calResult.sourceCat)," objects ")

In [ ]:
mytable = calResult.sourceCat.asAstropy()
mytable

### remove row with nan

In [ ]:
# remove row with nan
has_nan = np.zeros(len(mytable), dtype=bool)
for col in mytable.itercols():
    if col.info.dtype.kind == 'f':
        has_nan |= np.isnan(col)
mytable_no_nan = mytable[~has_nan]

### rename the columns

In [ ]:
# rename the columns
mytable_no_nan.rename_column('base_NaiveCentroid_x', 'Xpix')
mytable_no_nan.rename_column('base_NaiveCentroid_y', 'Ypix')
mytable_no_nan.rename_column('base_Blendedness_abs_child_instFlux', 'abs_chld_instFlux')
mytable_no_nan.rename_column('base_Blendedness_abs_parent_instFlux','abs_paren_instFlux')

In [ ]:
NN=len(mytable_no_nan)
values = mytable_no_nan['abs_chld_instFlux'].data
valmin = mytable_no_nan['abs_chld_instFlux'].min()
valmax = mytable_no_nan['abs_chld_instFlux'].max()
XX = mytable_no_nan['Xpix'].data
YY = mytable_no_nan['Ypix'].data
valmaxmin=valmax - valmin
index_star = mytable_no_nan['id'].data

In [ ]:
r = lambda x : (x - valmin)/valmaxmin

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=r(valmin), vmax=r(valmax))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(r(values), alpha=1)

In [ ]:
area = 200*r(values)
dx=-50
dy=100

In [ ]:
fig=plt.figure(figsize=(6,6))
ax=fig.add_subplot(1,1,1)
ax.set_title('X/Y locations of sources')
#ax.plot(mytable_no_nan['Xpix'],mytable_no_nan['Ypix'],'k.')
ax.scatter(mytable_no_nan['Xpix'],mytable_no_nan['Ypix'],s=area,color=all_colors)
ax.grid()
ax.set_xlim(0,ccd_xmax)
ax.set_ylim(0,ccd_ymax)

for idx in range(NN):
    ax.text(XX[idx]+dx, YY[idx]+dy, str(index_star[idx]))

In [ ]:
df_table = mytable_no_nan.to_pandas()
df_table.columns

In [ ]:
df = df_table[['id', 'coord_ra', 'coord_dec', 'Xpix','Ypix','abs_chld_instFlux'	,'abs_paren_instFlux']]
df

In [ ]:
the_max = df['abs_paren_instFlux'].describe()['max']
the_min = df['abs_paren_instFlux'].describe()['min']

In [ ]:
fig=plt.figure(figsize=(6,4))
ax=fig.add_subplot(1,1,1)
ax.hist(df['abs_paren_instFlux'].values,bins=50,range=(0,the_max));
ax.set_title("Flux of detected sources")
ax.set_xlabel('flux')
ax.set_ylabel('number of sources')